<a href="https://colab.research.google.com/github/anapitalai/gee/blob/main/geemap_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-dotenv

In [6]:
import geemap
import ee
from dotenv import load_dotenv
import os

In [7]:
dotenv_path = '/content/env/.env'
load_dotenv(dotenv_path=dotenv_path)

# Access the environment variables
project_name = os.getenv("project_name")

ee.Authenticate()
ee.Initialize(project=project_name)

In [8]:
Map = geemap.Map()
map=geemap.Map(center=[-6.677,146.677],zoom=12)
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
image = collection.median()
vis = {
'min': 0.0,
'max': 3000,
'bands': ['B4', 'B3', 'B2'],
}
Map.setCenter(146.677,-6.677, 12)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[-6.677, 146.677], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [ ]:
#l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
sentinel = ee.ImageCollection('COPERNICUS/S2_SR')
# dataset.bandNames().getInfo()
# image = dataset.select(['occurrence'])
# region = map.user_roi # Draw a polygon on the map
# if region is None:
#  region = ee.Geometry.BBox(-99.957, 46.8947, -99.278, 47.1531)
# vis_params = {'min': 0.0, 'max': 100.0, 'palette': ['ffffff', 'ffbbbb', '0000ff']}
# Map.addLayer(image, vis_params, 'Occurrence')
Map.addLayer(sentinel, {}, 'Sentinel-2', True, 0.5)

#Map.addLayer(l9, {}, 'Landsat-8', True, 0.5)
# Map.centerObject(region)
# Map.add_colorbar(vis_params, label='Water occurrence (%)', layer_name='Occurrence')


In [ ]:
df = geemap.image_histogram(
image,
region,
scale=30,
return_df=True,
)
df

,key,value
0,1,0.121569
1,6,0.137255
2,7,0.458824
3,8,0.258824
4,9,0.756863
...,...,...
72,95,2.854902
73,97,7.662745
74,98,47.972549
75,99,121.752941


from matplotlib import pyplot as plt
df['value'].plot(kind='hist', bins=20, title='value')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['value'].plot(kind='line', figsize=(8, 4), title='value')
plt.gca().spines[['top', 'right']].set_visible(False)

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:

dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
.filterDate('2024-05-01', '2025-01-11').filterMetadata('CLOUD_COVER', 'less_than', 10)

# Applies scaling factors.
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True)

dataset = dataset.map(applyScaleFactors)

visualization = {
    "bands": ['SR_B4', 'SR_B3', 'SR_B2'],
    "min": 0.0,
    "max": 0.3,
}

map.setCenter(146.99,-6.699, 8)

map.addLayer(dataset, visualization, 'True Color (432)')
map

NameError: name 'ee' is not defined

In [ ]:
dataset = ee.ImageCollection('SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL')
FalseColor = dataset.select(['N', 'G', 'B'])
FalseColorVis = {
    "min": 200.0,
    "max": 6000.0,
}
map.setCenter(146,-6.6, 15)
map.addLayer(FalseColor, FalseColorVis, 'False Color')
map

Map(bottom=120322.0, center=[-6.6, 146], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

In [ ]:
map=geemap.Map()
collection=(ee.ImageCollection("LANDSAT/LC08/C02/T1")
.filterBounds(ee.Geometry.Point([-6.6,146]))
.filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',90))

viz_params={
    'bands':['B4','B3','B2'],
    'min':0,
    'max':4000
}
map.addLayer(collection,viz_params)


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Press Ctrl+V in a new cell to paste it.

dataset = ee.Image('CGIAR/SRTM90_V4')
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation)
map.setCenter(146,6, 5)
map.addLayer(slope, {"min": 0, "max": 60}, 'slope')

Map(bottom=126995.0, center=[6, 146], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=S…

In [ ]:
m=geemap.Map()
m.split_map(left_layer="Esri.WorldImagery",right_layer="OpenTopoMap")
#m.add_basemap("OpenTopoMap")
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [ ]:
dem = ee.Image('USGS/SRTMGL1_003')
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
states = ee.FeatureCollection("TIGER/2018/States")

In [ ]:
dem_vis = {
'min': 0,
'max': 4000,
'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

landsat_vis = {
    'min': 20,
    'max': 200,
    'bands': ['B4', 'B3', 'B2']
}


In [ ]:
m.addLayer(dem, dem_vis, 'SRTM DEM', True, 0.5)
m.addLayer(landcover, {}, 'Land cover')
m.addLayer(landsat7, landsat_vis, 'Landsat 7')
m.addLayer(states, {}, "US States")
m

Map(bottom=579.0, center=[63.074865690586634, -2328.4944351540603], controls=(ZoomControl(options=['position',…